In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from fractions import Fraction
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")

from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit 
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

# parameters
customer_id = '5'
formatted_attribute = 'depth'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

strategy_version_input=702
attribute_id_input=2742

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'


def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
     
    
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))

def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

measure='This is a fraction: 100 1/64'
[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

from fractions import Fraction
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))
# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

df = query_from_file(file_name='./query/custom_fields.sql', params=params)
df.drop_duplicates(subset='external_id', inplace=True)

print(len(df))
# Drop duplicate external_ids
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# df=df[(df['external_id'].astype(str)!='200045713')&(df['external_id'].astype(str)!='200045715')&(df['external_id'].astype(str)!='200045714')]

# def regex_patterns(unit):
#     return fr'(?i)(\d+\.\d+.?.?.?{unit}\b)|(\d+.?\d+\/\d+.?.?.?{unit}\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?{unit}\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?{unit}\b)|()'
# print(regex_patterns('De?p?t?h?(?!ia)'))

from natsort import natsorted
regex_pattern=r'(?i)(\d+\.\d+.?.?.?De?p?t?h?(?!ia)\b)|(\d+.?\d+\/\d+.?.?.?D(?:epth)?(?!ia)\b)|((?<!\.)(?<!\/)\d+(?!\.\d)(?![^A-z]).?.?.?(?<!\d)De?p?t?h?(?!ia)(?!ay)\b)'                     
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[df['matches'].astype(str)!='[]']))
df['matches']=df['matches'].apply(lambda x: natsorted(x))

import time
today = time.strftime("%Y_%m_%d")
print(today)

678
[]
72
2023_04_25


# Depth

In [2]:
pat='''(?i)((?<=x.)\d+(?:\.\d+)?[^\d]{0,5}(?:L(?:ength)?.?x.?W(?:idth)?.?x.?D(?:epth)?))|((?<=x)\d+(?:\.\d+)?[^\d]{0,5}(?:L(?:ength)?.?x.?W(?:idth)?.?x.?D(?:epth)?))|()'''
df['LWD_name']=df['product_name'].apply(lambda x: re_extract(pat,str(x)))
df['LWD_desc']=df['long_desc'].apply(lambda x: re_extract(pat,str(x)))
df['LWD_custom']=df['custom_fields'].apply(lambda x: re_extract(pat,str(x)))
lwd=df[(df['LWD_name'].astype(str)!='[]')|(df['LWD_desc'].astype(str)!='[]')|(df['LWD_custom'].astype(str)!='[]')]
print(len(lwd))

pat='''(?i)((?<=x.)\d+(?:\.\d+)?[^\d]{0,5}.?x.?\d+(?:\.\d+)?[^\d]{0,5}(?<!cm)(?<!cm.)(?:L(?:ength)?.?x.?D(?:epth)?).?x.?H(?:eight)?)|((?<=x)\d+(?:\.\d+)?[^\d]{0,5}.?x.?\d+(?:\.\d+)?[^\d]{0,5}(?<!cm)(?<!cm.)(?:L(?:ength)?.?x.?D(?:epth)?).?x.?H(?:eight)?)|()'''
df['LDH_name']=df['product_name'].apply(lambda x: re_extract(pat,str(x)))
df['LDH_desc']=df['long_desc'].apply(lambda x: re_extract(pat,str(x)))
df['LDH_custom']=df['custom_fields'].apply(lambda x: re_extract(pat,str(x)))
ldh=df[(df['LDH_name'].astype(str)!='[]')|(df['LDH_desc'].astype(str)!='[]')|(df['LDH_custom'].astype(str)!='[]')]
print(len(ldh))

0
0


In [3]:
lwd[curation_col]=lwd['LWD_custom'].apply(lambda x: re.sub(r'''(?i)(\\?"\s?\(?\s?L\w*.?x.?W\w{0,6}.?x.?D\w{0,5})''',' in',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x))).apply(lambda x: re.sub(r'\,\s',',',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: '["'+'","'.join(natsorted(x))+'"]').apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x)))          
lwd[curation_col].explode().value_counts()

print(len(lwd))
rounding_inch_feet(lwd, curation_col,'a-eghj-mo-su-z')
lwd=lwd[lwd['rounding'].astype(str)=='[]']
print(len(lwd))



match_lwd=lwd[['external_id',curation_col]]

0
0


In [4]:
ldh[curation_col]=ldh['LDH_custom'].apply(lambda x: re.sub(r'''(?i)(\\?(?:"|'').?x.?\d+(?:\.\d+)?\\?(?:"|'')\s?\(?\（?\s?L\w*.?x.?D\w{0,6}.?x.?H\w{0,5})''',' in',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x))).apply(lambda x: re.sub(r'\,\s',',',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: '["'+'","'.join(natsorted(x))+'"]').apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x)))           
ldh[curation_col].explode().value_counts()

print(len(ldh))
rounding_inch_feet(ldh, curation_col,'a-eghj-mo-su-z')
ldh=ldh[ldh['rounding'].astype(str)=='[]']
print(len(ldh))

match_ldh=ldh[['external_id',curation_col]]

0
0


In [5]:
# pat='''(?i)([A-z]+.?x.?[A-z]+.?x.?[A-z]+)|()'''
# df['LDH_name']=df['product_name'].apply(lambda x: re_extract(pat,str(x)))
# df['LDH_desc']=df['long_desc'].apply(lambda x: re_extract(pat,str(x)))
# df['LDH_custom']=df['custom_fields'].apply(lambda x: re_extract(pat,str(x)))
# x=df[(df['LDH_name'].astype(str)!='[]')|(df['LDH_desc'].astype(str)!='[]')|(df['LDH_custom'].astype(str)!='[]')]
# print(len(x))

In [6]:
# df[df['units'].astype(str)!='[]']['units'].explode().value_counts()[0:500]

In [7]:
regex_pattern_blank=r'\d'
df['new_matches'] = df['matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

regex_pattern_blank=r'(?i)(\(?(?<![A-z])[A-z]\\?\s?x.{0,2}(?<![a-w])[A-z]\\?\s?x.{0,2}(?<![a-w])[A-z]\\?\s?\)?)|(\(?[A-z]\\?\s?x\\?\s?(?![A-z])[A-z]\\?\s?(?![A-z])\)?)'
df['units'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

df_new=df[(df['new_matches'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')]
df_new['matches']=df_new['matches'].apply(lambda x: re.sub(r'\\" D|"D|D|d',' in',str(x))).apply(lambda x: re.sub(r'\\" D|"D|\"xa0D|xa0D|"|epth','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
df_new['matches']=df_new['matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# df_new['matches']=df_new['matches'].apply(lambda x: round_string_float(x))
df_new['matches']=df_new['matches'].apply(lambda x: re.sub('\\\" D',' in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
df_new['matches']=df_new['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
df_new['matches']=df_new['matches'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|\)|inig','',str(x))).apply(lambda x: re.sub('  ',' ',str(x)))
del df_new['new_matches']
df_new['matches']=df_new['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
df_new['matches']=df_new['matches'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
df_new['matches']=df_new['matches'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x))))))))))))))))).apply(lambda x: re.sub(r'\_\s?in','in',str(x))).apply(lambda x: re.sub(r'(?<=\d)in',' in',str(x))) 
df_new['matches']=df_new['matches'].apply(lambda x: re.sub(r'((?<=\.\d)0(?!\d))|(\.0(?!\d))','',str(x)))

In [8]:
df_new[curation_col]=df_new['matches']
rounding_inch_feet(df_new, curation_col,'a-eghj-mo-su-z')
df_new=df_new[df_new['rounding'].astype(str)=='[]']

print(len(df_new))
rounding_inch_feet(df_new, curation_col,'a-eghj-mo-su-z')
df_new=df_new[df_new['rounding'].astype(str)=='[]']
print(len(df_new))

matches=df_new[['external_id',curation_col]]
print(len(matches))

36
36
36


# NA Values

In [9]:
regex_pattern_na=r'(?i)(\(?(?<![A-z])D\\?\s?x.{0,2}(?<![a-w])[A-z]\\?\s?x.{0,2}(?<![a-w])[A-z]\\?\s?\)?)|(\(?(?<![A-z])[A-z]\\?\s?x.{0,2}(?<![a-w])D\\?\s?x.{0,2}(?<![a-w])[A-z]\\?\s?\)?)|(\(?(?<![A-z])[A-z]\\?\s?x.{0,2}(?<![a-w])[A-z]\\?\s?x.{0,2}(?<![a-w])D\\?\s?\)?)|(\d+.{0,5}dia)|()'
df['na_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))
df_na=df[(df['na_matches'].astype(str)=='[]')&(df['matches'].astype(str)=='[]')]
print(len(df_na))

df_na[curation_col]='n/a'
matches_na=df_na[['external_id',curation_col]]

431


# W X D X H

In [10]:
pat=r'(?i)((?<=x)\s?\d+(?:\.\d)?\\{0,3}(?:"|''|“)\s?(?=x).{0,12}(?=\(W\s?x\s?D\s?x\s?\w\)))|()'
df['trips'] = df['long_desc'].apply(lambda x: re_extract(pat, x))

pat=r'(?i)(W x D x H)|()'
df['WDH'] = df['long_desc'].apply(lambda x: re_extract(pat, x))

df_wdh=df[(df['na_matches'].astype(str)!='[]')&(df['matches'].astype(str)=='[]')&(df['WDH'].astype(str)!='[]')]
df_wdh['Q:depth']=df_wdh['trips'].apply(lambda x: re.sub(r'(?i)(\s?x.{0,15}(?=\]))|(\s?x.{0,15}(?=\,))','',str(x))).apply(lambda x: re.sub(r'\\{1,3}"',' in',str(x))).apply(lambda x: re.sub(r'''((?<=)\s?'\s?)|(?<!')(?<!")(?=\,)|(?<=\[)(?!')(?!")|(?<!')(?<!")(?=\])|(?:''|“)|("")''','"',str(x)))                        

df_wdh[curation_col]=df_wdh[curation_col].apply(lambda x: re.sub(r'(?i)(?<=\d)(?="\])',' in',str(x)))
df_wdh=df_wdh[df_wdh['trips'].astype(str)!='[]']
print(len(df_wdh))
curate(df_wdh,curation_col, 'depth')

print(len(df_wdh))
rounding_inch_feet(df_wdh, curation_col,'a-eghj-mo-su-z')
df_wdh=df_wdh[df_wdh['rounding'].astype(str)=='[]']
print(len(df_wdh))

match_wdh=df_wdh[['external_id',curation_col]]
rounding_inch_feet(df_wdh, curation_col,'a-eghj-mo-su-z')

35
Repeated External IDs: []
35
35


Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, custom_fields, value, matches, LWD_name, LWD_desc, LWD_custom, LDH_name, LDH_desc, LDH_custom, new_matches, units, na_matches, trips, WDH, Q:depth, ones, twos, threes, fours, fives, rounding]
Index: []

In [11]:
regex_pattern=r'(?i)(\d+.?\d?\/?\d*\\\"\s?H\b)|(\d*\.\d*\\\"\s?H\b)|()'
df['matchez'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

In [12]:
regex_pattern_blank=r'\d'
df['new_matches'] = df['matchez'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_len=df[(df['matchez'].astype(str)!='[]')&(df['new_matches'].astype(str)!='[]')&(df['na_matches'].astype(str)!='[]')&(df['matches'].astype(str)=='[]')&(df['WDH'].astype(str)!='[]')]
print(len(df_len))
# df_len=df_len[df_len['new_matches'].astype(str)!='[]']

0


In [13]:
regex_pattern_comma=r','
df_len['comma_matches'] = df_len['matchez'].apply(lambda x: re_extract(regex_pattern_comma, str(x)))
natural_sort=df_len[df_len['comma_matches'].astype(str)!='[]']
natural_sort['matches']=natural_sort['matchez'].apply(lambda x: sorted(x, key=natural_keys))

In [14]:
natural_sort['matches']=natural_sort['matches'].apply(lambda x: re.sub(r'\\" W|"W',' in',str(x))).apply(lambda x: re.sub(r'\\" W|"W|\"xa0W|xa0W|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
# natural_sort['matches']=[list(set(sublist)) for sublist in natural_sort['matches']]
natural_sort['matches']=natural_sort['matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
natural_sort['matches']= natural_sort['matches'].apply(lambda x: sorted(x, key=natural_keys))
natural_sort['matches']=natural_sort['matches'].apply(lambda x: re.sub("\['15.9, 15.9'\]","['15.9']",str(x)))
natural_sort['matches']=natural_sort['matches'].apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub(",",'","',str(x))).apply(lambda x: re.sub('" ','"',str(x))).apply(lambda x: re.sub('","',' in","',str(x))).apply(lambda x: re.sub('"]',' in"]',str(x))).apply(lambda x: re.sub('in in','in',str(x))).apply(lambda x: re.sub(' H','',str(x)))                                                                          
# natural_sort['matches']=[list(set(sublist)) for sublist in natural_sort['matches']]

In [15]:
natural_sort[curation_col]=natural_sort['matches']


print(len(natural_sort))
rounding_inch_feet(natural_sort, curation_col,'a-eghj-mo-su-z')
natural_sort=df_wdh[df_wdh['rounding'].astype(str)=='[]']
print(len(df_wdh))


matches_natural=natural_sort[['external_id',curation_col]]


print(len(matches_natural))

0
35
35


In [16]:
df_news=df_len[df_len['comma_matches'].astype(str)=='[]']
df_news['matches']=df_news['matchez'].apply(lambda x: re.sub(r'\\" H|"H|\"xa0H|xa0H|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
df_news['matches']=df_news['matches'].apply(lambda x: remove_frac(remove_fraction(x)))
df_news['matches']=df_news['matches'].apply(lambda x: round_string_float(x))

In [17]:
df_news['matches']=df_news['matches'].apply(lambda x:str(x)+' in')

In [18]:
df_news[curation_col]=df_news['matches']

print(len(df_news))
rounding_inch_feet(df_news, curation_col,'a-eghj-mo-su-z')
df_news=df_news[df_news['rounding'].astype(str)=='[]']
print(len(df_news))

matches_new=df_news[['external_id',curation_col]]
print(len(matches_new))
print(len(matches_natural))

0
0
0
35


In [19]:
rounding_inch_feet(df_new, curation_col,'a-eghj-mo-su-z')

Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, custom_fields, value, matches, LWD_name, LWD_desc, LWD_custom, LDH_name, LDH_desc, LDH_custom, units, Q:depth, rounding]
Index: []

In [20]:
print(len(df))
print(len(matches_na)+len(matches)+len(match_wdh))

678
502


In [21]:
# df_wdh[0:500]#[df_wdh['external_id'].astype(str)=='200186570']
# df_new

In [22]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [23]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute,matches_na,today)

In [24]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute,matches,today)

In [25]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}_matche_wdh.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute,match_wdh,today)

In [26]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}match_lwd.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute,match_lwd,today)

In [27]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}match_lhd.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute,match_ldh,today)